In [5]:
import findspark
findspark.init('/home/caio/spark-2.1.0-bin-hadoop2.7/')
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('aggs').getOrCreate()

In [4]:
df = spark.read.csv('/home/caio/Desktop/Spark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/sales_info.csv',inferSchema=True,header=True)

AnalysisException: 'Path does not exist: file:/home/caio/Desktop/Spark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/sales_info.csv;'

In [ ]:
df.show()

In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [6]:
data = df.toPandas()

In [10]:
import pandas as pd

In [12]:
data['Company']

0     GOOG
1     GOOG
2     GOOG
3     MSFT
4     MSFT
5     MSFT
6       FB
7       FB
8     APPL
9     APPL
10    APPL
11    APPL
Name: Company, dtype: object

In [13]:
df2 = pd.get_dummies(data['Company'])


In [15]:
data[df2.columns] = df2

In [16]:
data

,Company,Person,Sales,APPL,FB,GOOG,MSFT
0,GOOG,Sam,200.0,0,0,1,0
1,GOOG,Charlie,120.0,0,0,1,0
2,GOOG,Frank,340.0,0,0,1,0
3,MSFT,Tina,600.0,0,0,0,1
4,MSFT,Amy,124.0,0,0,0,1
5,MSFT,Vanessa,243.0,0,0,0,1
6,FB,Carl,870.0,0,1,0,0
7,FB,Sarah,350.0,0,1,0,0
8,APPL,John,250.0,1,0,0,0
9,APPL,Linda,130.0,1,0,0,0


In [10]:
df.groupBy("Company").count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [12]:
df.groupBy("Company").agg({'Sales':'sum'}).show()

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [14]:
from pyspark.sql.functions import countDistinct,avg,stddev

In [17]:
df.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [18]:
df.select(stddev('Sales')).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [19]:
from pyspark.sql.functions import format_number

In [20]:
sales_std = df.select(stddev('Sales').alias('Std'))

In [23]:
sales_std.select(format_number('Std',2).alias('Std')).show()

+------+
|   Std|
+------+
|250.09|
+------+



In [24]:
df.orderBy("Sales").show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [27]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



In [28]:
df2 = spark.read.csv('/home/caio/Desktop/Spark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/ContainsNull.csv',header=True,inferSchema=True)

In [29]:
df2.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [31]:
df2.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [33]:
df2.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [34]:
df2.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [38]:
df2.na.fill('No Name',subset=['Name']).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [39]:
from pyspark.sql.functions import mean

In [41]:
mean_val = df2.select(mean(df2['Sales'])).collect()

In [44]:
mean_sales = mean_val[0][0]

In [46]:
df2.na.fill(mean_sales,['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [47]:
df2.na.fill(df2.select(mean(df2['Sales'])).collect()[0][0],['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

